# Q1. Refactoring

Before we can start coverting our code with tests, we need to 
refactor it. We'll start by getting rid of all the global variables. 

* Let's create a function `main` with two parameters: `year` and
`month`.
* Move all the code (except `read_data`) inside `main`
* Make `categorical` a parameter for `read_data` and pass it inside `main`

Now we need to create the "main" block from which we'll invoke
the main function. How does the `if` statement that we use for
this looks like? 


Hint: after refactoring, check that the code still works. Just run
it e.g. for Feb 2021 and see if it finishes successfully. 

To make it easier to run it, you can write results to your local
filesystem.

In [14]:
! python batch.py "2021" "03"

starting
predicted mean duration: 16.298821614015107
Saved on taxi_type=fhv_year=2021_month=03.parquet
ending


In [15]:
! ls

batch.py
data
homework.ipynb
model.bin
taxi_type=fhv_year=2021_month=03.parquet
tests


As suggested, the `taxi_type=fhv_year=2021_month=03.parquet`is there after executing the refactored code.

 How does the if statement that we use for this looks like? 
 
 `if __name__ == "__main__":`

# Q2. Installing pytest

In [16]:
! pipenv install --dev pytest

Installing pytest...
Installing dependencies from Pipfile.lock (2e21b6)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.


Creating a virtualenv for this project...

Pipfile: C:\Users\cami1\Escritorio\Github\mlops-zoomcamp\00-homework\06-best-practices\notebooks\Pipfile

Using C:/Users/cami1/AppData/Local/Programs/Python/Python310/python.exe (3.10.0) to create virtualenv...


[    ] Creating virtual environment..
[=   ] Creating virtual environment..
[==  ] Creating virtual environment..
[=== ] Creating virtual environment..
[ ===] Creating virtual environment..
[  ==] Creating virtual environment..
[   =] Creating virtual environment..
[    ] Creating virtual environment..
[   =] Creating virtual environment..
[  ==] Creating virtual environment..
[ ===] Creating virtual environment..
[====] Creating virtual environment..
[=== ] Creating virtual environment..
[==  ] Creating virtual environment..
[=   ] Creating virtual environment..
[    ] Creating virtual environment..
[=   ] Creating virtual environment..
[==  ] Creating virtual environment..
[=== ] Creating virtual environment..
[ ===] Creating virtua

Next, create a folder tests and create two files. One will be the file with tests. We can name if test_batch.py.

What should be the other file?

Hint: to be able to test batch.py, we need to be able to import it. Without this other file, we won't be able to do it.

In [18]:
! ls tests

__init__.py
test_batch.py


# Q3. Writing first unit test

How many rows should be there in the expected dataframe?

In [10]:
! pytest tests\test_batch.py

============================= test session starts =============================
platform win32 -- Python 3.10.0, pytest-7.1.2, pluggy-1.0.0
rootdir: c:\Users\cami1\Escritorio\Github\mlops-zoomcamp\00-homework\06-best-practices\notebooks
collected 1 item

tests\test_batch.py .                                                    [100%]

============================== 1 passed in 1.05s ==============================


In [12]:
! python tests\test_batch.py

   PUlocationID  DOlocationID     pickup_datetime    dropOff_datetime
2           1.0           1.0 2021-01-01 01:02:00 2021-01-01 01:02:50
0           NaN           NaN 2021-01-01 01:02:00 2021-01-01 01:10:00
1           1.0           1.0 2021-01-01 01:02:00 2021-01-01 01:10:00
3           1.0           1.0 2021-01-01 01:02:00 2021-01-01 02:02:01
# rows: 4


4 rows

# Q4.  Mocking S3 with Localstack

In [58]:
! pip install pipenv

  Attempting uninstall: pip
    Found existing installation: pip 21.2.2
    Uninstalling pip-21.2.2:
      Successfully uninstalled pip-21.2.2


In [60]:
! pipenv install scikit-learn==1.0.2

Installing scikit-learn==1.0.2...
Installing dependencies from Pipfile.lock (9791c1)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.


Creating a virtualenv for this project...

Pipfile: C:\Users\cami1\Escritorio\Github\mlops-zoomcamp\00-homework\04-deployment\notebooks\Pipfile

Using C:/Users/cami1/AppData/Local/Programs/Python/Python310/python.exe (3.10.0) to create virtualenv...


[    ] Creating virtual environment..
[=   ] Creating virtual environment..
[==  ] Creating virtual environment..
[=== ] Creating virtual environment..
[ ===] Creating virtual environment..
[  ==] Creating virtual environment..
[   =] Creating virtual environment..
[    ] Creating virtual environment..
[   =] Creating virtual environment..
[  ==] Creating virtual environment..
[ ===] Creating virtual environment..
[====] Creating virtual environment..
[=== ] Creating virtual environment..
[==  ] Creating virtual environment..
[=   ] Creating virtual environment..
[    ] Creating virtual environment..
[=   ] Creating virtual environment..
[==  ] Creating virtual environment..
[=== ] Creating virtual environment..
[ ===] Creating virtual en

In [70]:
import json
  
with open('Pipfile.lock') as f:
    data = json.load(f)

In [73]:
library = "scikit-learn"
hash = data["default"][library]["hashes"][0]
version = data["default"][library]["version"]
print(f"The library {library} with the version {version} has as first hash {hash}")

The library scikit-learn with the version ==1.0.2 has as first hash sha256:08ef968f6b72033c16c479c966bf37ccd49b06ea91b765e1cc27afefe723920b


What's the first hash for the Scikit-Learn dependency?

**sha256:08ef968f6b72033c16c479c966bf37ccd49b06ea91b765e1cc27afefe723920b**

# Q5. Parametrize the script

In [26]:
! python ../src/predict.py -y "2021" -m "03"

16.298821614015107


What's the mean predicted duration?

**16.298**

# Q6. Docker container

In [18]:
! cat Dockerfile

FROM agrigorev/zoomcamp-model:mlops-3.9.7-slim

RUN pip install -U pip
RUN pip install pipenv 
RUN pip install pipenv 

WORKDIR /app
COPY [ "Pipfile", "Pipfile.lock", "./" ]
RUN pipenv install --system

COPY [ "predict.py", "predict.py"]

CMD [ "python", "predict.py", "-y", "2021", "-m", "04" ]



In [20]:
! docker run --mount type=bind,source="$(cd ../ && pwd)"/src/data,target=/app/data -it mlopszoomcamp

9.967573179784523


Now run the script with docker. What's the mean predicted duration for April 2021?

**9.96**